In [1]:
import numpy as np

In [22]:
A=np.array([[1,2,3,4],[2,1,4,3],[3,4,2,1],[4,2,3,1]],dtype=float)
B=np.random.rand(4,4)
C=np.array([[1,4,4,3],[5,2,4,2],[1,1,2,1],[3,2,1,2]],dtype=float)
sys=np.array([[1,2,2],[3,2,1],[1,3,2]], dtype=float)
b=np.array([[11],[10],[10]], dtype=float)
Tester=sys

# Naive LU Decomp

In [23]:
def myLU(A):
    m,n=A.shape #Get row/col
    U=A.copy() #Make Changable Copy
    L=np.eye(m) #Blank canvas for L
    if(m!=n): #Test Square
        print("NonSquare input")
        return 1
    for c in range(0,n-1): #Each Column but the last 
        for r in range(int(c+1),m): #Sub Rows based on column
            d=U[r,c]/U[c,c] #Get Scalar from subrow to cancel values
            L[r,c]=d # Add to L matrix in relevant coordinate
            U[r,:]=U[r,:]-d*U[c,:] # Update row by cancelling lower triangular values
    return L,U #Return LU decomposition

In [24]:
L,U=myLU(Tester)
print("A=\n",Tester,"\n")
print("L=\n",L,"\n")
print("U=\n",U,"\n")
print("Recombining:")
print(np.matmul(L,U))
print("-"*65)
print("Error=\n",Tester-np.matmul(L,U))

A=
 [[1. 2. 2.]
 [3. 2. 1.]
 [1. 3. 2.]] 

L=
 [[ 1.    0.    0.  ]
 [ 3.    1.    0.  ]
 [ 1.   -0.25  1.  ]] 

U=
 [[ 1.    2.    2.  ]
 [ 0.   -4.   -5.  ]
 [ 0.    0.   -1.25]] 

Recombining:
[[1. 2. 2.]
 [3. 2. 1.]
 [1. 3. 2.]]
-----------------------------------------------------------------
Error=
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


# Partial Pivoting

In [49]:
def myLUPartial(A):
    m,n=A.shape
    P = np.arange(m)
    flop=np.zeros((m,m))
    U=A.copy()
    L=np.eye(m)
    if(m!=n):
        print("NonSquare input")
        return 1
    for c in range(0,n-1):
        V=U.copy()
        O=P.copy()
        M=np.argmax(np.abs(U[c:,c]))
        if(M+1!=c):
            U[M+c,:]=V[c,:]
            U[c,:]=V[M+c,:]
            P[M+c]=O[c]
            P[c]=O[M+c]
        for r in range(int(c+1),m):
            d=U[r,c]/U[c,c]
            L[r,c]=d
            U[r,:]=U[r,:]-d*U[c,:]
    for i in range(0,n):
        flop[P[i],i]=1
    return L,U, flop

In [63]:
L,U, flop=myLUPartial(sys)
print("A=\n",sys,"\n")
print("L=\n",L,"\n")
print("U=\n",U,"\n")
print("Recombining:")
print(np.matmul(L,U))
print("flop")
print(flop)
print("Swapping")
print(np.matmul(flop,np.matmul(L,U)))
print("-"*65)
print("Error=\n",sys-np.matmul(flop,np.matmul(L,U)))

A=
 [[1. 4. 4. 3.]
 [5. 2. 4. 2.]
 [1. 1. 2. 1.]
 [3. 2. 1. 2.]] 

L=
 [[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.2         0.16666667  1.          0.        ]
 [ 0.6         0.22222222 -3.16666667  1.        ]] 

U=
 [[5.         2.         4.         2.        ]
 [0.         3.6        3.2        2.6       ]
 [0.         0.         0.66666667 0.16666667]
 [0.         0.         0.         0.75      ]] 

Recombining:
[[5. 2. 4. 2.]
 [1. 4. 4. 3.]
 [1. 1. 2. 1.]
 [3. 2. 1. 2.]]
flop
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Swapping
[[1. 4. 4. 3.]
 [5. 2. 4. 2.]
 [1. 1. 2. 1.]
 [3. 2. 1. 2.]]
-----------------------------------------------------------------
Error=
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# Solver

In [80]:
def solveY(L,b):
    m,n=L.shape
    y=np.zeros(m)
    for i in range(0,m):
        print(np.dot(L[i,:],y))
        y[i]=(b[i]-np.dot(L[i,:],y))/L[i,i]
        print(y)
        print(L[i,:])
        print("T")
    return y

def solveX(U,y):
    m,n=L.shape
    x=np.zeros(m)
    for i in range(n-1,-1,-1):
        x[i]=(y[i]-np.dot(L[i,:],y))/U[i,i]
    return x

def myLUSolver(A,b):
    L,U,f = myLUPartial(A)
    print("Ly=b")
    print(L)
    print(np.matmul(f,b))
    y=solveY(L,np.matmul(f,b))
    print(y)
    x=solveX(U,y)
    print(x)

In [81]:
#sys=np.array([[1,2,2],[3,2,1],[1,3,2]], dtype=float)
#b=np.array([[11],[10],[13]], dtype=float)
sys=np.array([[1,4,4,3],[5,2,4,2],[1,1,2,1],[3,2,1,2]], dtype=float)
b=np.array([[33],[29],[13],[18]], dtype=float)
myLUSolver(sys,b)

Ly=b
[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.2         0.16666667  1.          0.        ]
 [ 0.6         0.22222222 -3.16666667  1.        ]]
[[29.]
 [33.]
 [13.]
 [18.]]
0.0
[29.  0.  0.  0.]
[1. 0. 0. 0.]
T
5.800000000000001
[29.  27.2  0.   0. ]
[0.2 1.  0.  0. ]
T
10.333333333333334
[29.         27.2         2.66666667  0.        ]
[0.2        0.16666667 1.         0.        ]
T
15.0
[29.         27.2         2.66666667  3.        ]
[ 0.6         0.22222222 -3.16666667  1.        ]
T
[29.         27.2         2.66666667  3.        ]
[  0.          -1.61111111 -15.5        -20.        ]
